## Data Quality Issues

In [1]:
import json
import gzip
import pandas as pd
import warnings
from functools import partial
import uuid
from pandas import json_normalize
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)

### Issue 1: Users data contains duplicates: users data contain 283 duplicate values

In [2]:
#read user data
filename = 'users.json.gz'  

json_content = []
with gzip.open(filename , 'rb') as gzip_file:
    for line in gzip_file:
        line = line.rstrip()
        if line:  
            obj = json.loads(line)
            json_content.append(obj)

data=json.dumps(json_content)
df_users = pd.read_json(data, orient ='records')
df_users['_id']=df_users['_id'].apply(lambda x: x['$oid'])

In [3]:
df_users[df_users['_id']=='5ff1e194b6a9d73a3a9f1052']

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
5,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
8,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
10,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
12,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
14,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
15,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [4]:
# users data contain 283 duplicate values
print(df_users._id.duplicated().sum())
df_users[df_users._id.duplicated()]

283


,_id,active,createdDate,lastLogin,role,signUpSource,state
1,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
5,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
8,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN


### Issue 2: Receipts data contains lots of null values

In [5]:
#read receipt data
filename = 'receipts.json.gz'  # Sample file.

json_content = []
with gzip.open(filename , 'rb') as gzip_file:
    for line in gzip_file:
        line = line.rstrip()
        if line:  
            obj = json.loads(line)
            json_content.append(obj)

data=json.dumps(json_content)

#clean receipt data
df_receipt = pd.read_json(data, orient ='records')

#get the records where its receipitemlist doesn't contain null values
df_receipt['is_emptyitemList']=df_receipt['rewardsReceiptItemList'].isnull()
temp=df_receipt[['_id','rewardsReceiptItemList','is_emptyitemList','rewardsReceiptStatus']]
itemList=temp[temp['is_emptyitemList']==False]

#get all the columns that appear in the 'rewardsReceiptItemList'
keyList=list(itemList['rewardsReceiptItemList'][0][0].keys())
for row in itemList['rewardsReceiptItemList']:
    rowList=row[0].keys()
    for key in rowList:
        if key not in keyList:
            keyList.append(key)


df_all= pd.DataFrame(columns = keyList)

# convert the 'rewardsReceiptItemList' which is in json format to pandas dataframe
itemList_index=itemList.index.tolist()
receiptList=[]
for index in itemList_index:
    df=json_normalize(json_content[index]['rewardsReceiptItemList'])
    df_all=pd.concat([df_all,df])


In [6]:
df_all

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity,originalMetaBriteBarcode,originalMetaBriteDescription,pointsNotAwardedReason,pointsPayerId,rewardsGroup,rewardsProductPartnerId,brandCode,competitorRewardsGroup,discountedItemPrice,originalReceiptItemText,itemNumber,needsFetchReviewReason,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,userFlaggedDescription,deleted,priceAfterCoupon,metabriteCampaignId,originalFinalPrice,originalMetaBriteItemPrice
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5,4011,True,26.00,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1,028400642255,True,10.00,1.0,NaN,NaN,Action not allowed for user and CPG,5332f5fbe4b03c9a25efd0ba,DORITOS SPICY SWEET CHILI SINGLE SERVE,5332f5fbe4b03c9a25efd0ba,NaN,NaN,NaN,NaN,NaN,USER_FLAGGED,NaN,NaN,NaN,NaN,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,26.00,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4,4011,True,28.00,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,thindust summer face mask - sun protection nec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN,NaN,NaN
0,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,mueller austria hypergrind precision electric ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN,NaN,NaN
1,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,thindust summer face mask - sun protection nec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN,NaN,NaN
0,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,mueller austria hypergrind precision electric ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN,NaN,NaN


In [7]:
df_all.isnull().sum()

barcode                               3851
description                            381
finalPrice                             174
itemPrice                              174
needsFetchReview                      6128
partnerItemId                            0
preventTargetGapPoints                6583
quantityPurchased                      174
userFlaggedBarcode                    6604
userFlaggedNewItem                    6618
userFlaggedPrice                      6642
userFlaggedQuantity                   6642
originalMetaBriteBarcode              6870
originalMetaBriteDescription          6931
pointsNotAwardedReason                6601
pointsPayerId                         5674
rewardsGroup                          5210
rewardsProductPartnerId               4672
brandCode                             4341
competitorRewardsGroup                6666
discountedItemPrice                   1172
originalReceiptItemText               1181
itemNumber                            6788
needsFetchR